<img src="data/images/lecture-notebook-header.png" />

# Classification & Regression I: Cross Validation

Cross-validation is a widely used technique in machine learning for evaluating the performance of a predictive model and estimating its generalization ability. It helps assess how well a model will perform on unseen data. Most classifiers or regressors feature a set of hyperparameters (e.g., the k in KNN) that can significantly affect the results. To find the best parameter settings, we have to train and evaluate for different parameter values.

However, this evaluation of finding the best parameter values cannot be done using the test set. The test set has to be unseen using the very end for the final evaluation (once the hyperparameters have been fixed). Using the test set to tune the hyperparameters means that the test set has affected the training process.

The process of cross-validation involves splitting the available dataset into multiple subsets or "folds." One of the folds is used as the validation set, while the remaining folds are used for training the model. This process is repeated multiple times, with each fold serving as the validation set in a different iteration.

Here's a step-by-step explanation of a common cross-validation procedure called "k-fold cross-validation":

* The dataset is divided into k subsets or folds of approximately equal size.
* The model is trained k times, each time using k-1 folds as the training data and one fold as the validation data.
* The performance of the model is evaluated on each validation set, typically by calculating a performance metric such as accuracy, precision, recall, or F1 score.
* The performance scores obtained from each fold are averaged to get an overall performance estimate of the model.

The value of k can vary, but common choices include 5-fold or 10-fold cross-validation. Generally, a larger value of k leads to a more robust performance estimation but increases the computational cost. Once the model is validated using cross-validation and its performance is satisfactory, it can be trained on the entire dataset (without the need for validation sets) and used for making predictions on new, unseen data.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Make all Required Imports

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing
from sklearn.model_selection import KFold

from sklearn.metrics import classification_report, f1_score, mean_squared_error
from sklearn.model_selection import cross_val_score

---

## KNN Classification of Pima Dataset

The [Pima Indians Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) is a well-known dataset frequently used in machine learning and data mining research. It contains information about a group of Pima Indian women from Arizona, USA, and their potential risk of developing diabetes. The dataset is commonly used for classification tasks, aiming to predict whether a person has diabetes or not based on various features.

Here are the details of the Pima diabetes dataset:

* Number of instances: 768
* Number of attributes: 8
* Target variable: Outcome (0 for non-diabetic, 1 for diabetic)

The attributes or features in the dataset are:

* Pregnancies: Number of times pregnant.
* Glucose: Plasma glucose concentration after a 2-hour oral glucose tolerance test.
* Blood Pressure: Diastolic blood pressure (mm Hg).
* Skin Thickness: Triceps skinfold thickness (mm).
* Insulin: 2-Hour serum insulin (mu U/ml).
* BMI: Body mass index (weight in kg / (height in m)^2).
* Diabetes Pedigree Function: Diabetes pedigree function, which provides an estimation of the genetic influence.
* Age: Age in years.

This is a convenient dataset since all attributes for the clustering are numerical, and we can use the default Euclidean distance as a similarity measure. First, as usual, we read the dataset from the file into a `pandas` DataFrame.

In [ ]:
df_diabetes = pd.read_csv('data/datasets/diabetes/diabetes.csv')

# The rows are sorted, so let's shuffle them
df_diabetes = df_diabetes.sample(frac=1).reset_index(drop=True)

# Show the first 5 columns
df_diabetes.head()

### Create Training and Test Data

In [ ]:
# Convert data to numpy arrays
X = df_diabetes[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']].to_numpy()
y = df_diabetes[['Outcome']].to_numpy().squeeze()

# Let's go for a 80%/20% split -- you can change the value anf see its effects
train_test_ratio = 0.80

# Calculate the size of the training data (the size of the dest data is also implicitly given)
train_set_size = int(train_test_ratio * len(X))

# Split data and labels into training and test data with respect to the size of the test data
X_train, X_test = X[:train_set_size], X[train_set_size:]
y_train, y_test = y[:train_set_size].squeeze(), y[train_set_size:].squeeze()

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))

### Standardize Data

Although we have only numerical values as input attributes there magnitudes and ranges differ noticeable. It's therefore a good idea to normalize/standardize the data. As usual, `scitkit-learn` makes it very convenient by providing a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) (among other methods for normalization and standardization).

**Important:** We have fit the scaler on the training data `X_train` only (fitting here means to calculate the mean and standard deviation)! If we would use the `X` for that, then this would include the test data `X_test`. In this case, the test data would affect the transformation and training steps. However, `X_test` has to remain truly "unseen" until the very end.

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
#scaler = preprocessing.StandardScaler().fit(X)  # WRONG!!!

X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

### Train and Test KNN Classifier Using Cross-Validation

#### Semi-Manually K-Fold Validation

We first utilize `scikit-learn`'s  [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) to split the training data into $k$ folds (here, $k=10$). The [`KFold.split()`] method generates the folds and allows to loop over all combinations of training and validation folds. Each combination contains $k-1$ training folds and 1 validation fold. For each combination we can retrain and validate the classifier.

In [ ]:
# Initialize the best f1-score and respective k value
k_best, f1_best = None, 0.0

# Loop over a range of values for setting k
for k in tqdm(range(1, 20)):

    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    f1_scores = []
    for train_index, val_index in kf.split(X_train_transformed):
        
        # Create the next combination of training and validation folds
        X_trn, X_val = X_train_transformed[train_index], X_train_transformed[val_index]
        y_trn, y_val = y_train[train_index], y_train[val_index]
    
        # Train the classifier for the current training folds
        classifier = KNeighborsClassifier(n_neighbors=k).fit(X_trn, y_trn)
        
        # Predict the labels for the validation fold
        y_pred = classifier.predict(X_val)

        # Calculate the f1-score for the validation fold
        f1_scores.append(f1_score(y_val, y_pred, average='macro'))
        
    # Calculate f1-score for all fold combination as the mean over all scores
    f1_fold_mean = np.mean(f1_scores)
    
    # Keep track of the best f1-score and the respective k value
    if f1_fold_mean > f1_best:
        k_best, f1_best = k, f1_fold_mean
        
        
print('The best f1-score was {:.3f} for k={}'.format(f1_best, k_best))

### Automatic Cross-Validation

`scikit-learn` provides the even more convenient method [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) that does the generation of folds and splitting them into training folds and validation folds, as well as the training of a classifier for all folds.

In [ ]:
# Initialize the best f1-score and respective k value
k_best, f1_best = None, 0.0

# Loop over a range of values for setting k
for k in tqdm(range(1, 20)):
    
    # Specfify type of classifier
    classifier = KNeighborsClassifier(n_neighbors=k)
    
    # perform cross validation (here with 10 folds)
    # f1_scores is an array containg the 10 f1-scores
    f1_scores = cross_val_score(classifier, X_train_transformed, y_train, cv=10, scoring='f1_macro')
    
    # Calculate the f1-score for the current k value as the mean over all 10 f1-scores
    f1_fold_mean = np.mean(f1_scores)
    
    # Keep track of the best f1-score and the respective k value
    if f1_fold_mean > f1_best:
        k_best, f1_best = k, f1_fold_mean
  

print('The best f1-score was {:.3f} for a k={}'.format(f1_best, k_best))

**Side note:** The 2 previous code cell might yiel different results for the best f1-score and the corresponding best value for $k$. This is because the splitting into folds has a random component.

## Final Evaluation on Test Data

Now that we have identified the best value for $k$, we can perform the final evaluation using the test data. We can now also use the fill training data, and don't need to split it into any folds.

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=k_best).fit(X_train_transformed, y_train)

y_pred = classifier.predict(X_test_transformed)

f1_final = f1_score(y_test, y_pred, average='macro')

print('The final f1-score of the KNN classifier (k={}) is: {:.3f}'.format(k_best, f1_final))
        

This final score is the one to report when quantifying the quality of the classifier.

## Summary

Cross-validation plays a crucial role in machine learning as it serves two primary purposes: model evaluation and hyperparameter tuning. By evaluating the model's performance using cross-validation, we can estimate its ability to generalize to unseen data and compare it against other models or algorithms. This evaluation helps in selecting the best-performing model for deployment.

One of the key challenges in machine learning is overfitting, where a model learns to perform well on the training data but fails to generalize to new data. Cross-validation helps mitigate this issue by providing a more reliable estimate of the model's performance. By repeatedly training and evaluating the model on different subsets of the data, cross-validation helps identify models that generalize well.

Another challenge in machine learning is the selection of optimal hyperparameters. Hyperparameters are parameters that are not learned from the data but are set manually by the user, such as the learning rate in neural networks or the depth of a decision tree. Cross-validation can be used to tune these hyperparameters by evaluating the model's performance with different combinations of hyperparameter values. This helps in finding the best set of hyperparameters that optimize the model's performance.

However, cross-validation also presents certain challenges. One such challenge is the potential for data leakage. Data leakage occurs when information from the validation set inadvertently influences the model during training, leading to overly optimistic performance estimates. Care must be taken to ensure that the validation set remains independent and untouched during the training process.

Another challenge is computational cost. Cross-validation involves training and evaluating the model multiple times, which can be time-consuming and computationally expensive, especially for large datasets or complex models. This challenge can be mitigated by using techniques like stratified sampling or parallel computing to improve efficiency.

In summary, cross-validation is a vital technique in machine learning for model evaluation and hyperparameter tuning. It helps estimate a model's generalization performance, identify overfitting, and select optimal hyperparameters. While challenges like data leakage and computational cost exist, proper implementation and careful consideration of these challenges can ensure the effectiveness and reliability of cross-validation in improving model performance.